In [5]:
import mysql.connector
import pycountry
import plotly.express as px
import pandas as pd
import numpy as np

mydb = mysql.connector.connect(
  host="127.0.0.1",
  user="root",
  password="qwerty123",
  database="cs306"
)

mycursor = mydb.cursor()
mycursor.execute(
    '''
    Select pfr_.Country, AVG(pfr_.PFR) AS PFR_average
        FROM cs306.pfr pfr_
        GROUP BY pfr_.Country;
    '''
)

df = pd.DataFrame(columns=["Country", "iso_alpha", "PFR_average"])

for (Country, PFR_average) in mycursor:
    if (Country == 'Russia'):
        c = pycountry.countries.get(name='Russian Federation')
        tmp = pd.DataFrame(
            [{"Country": "Russian Federation", "iso_alpha": c.alpha_3, "PFR_average": float(PFR_average) }])
        df = pd.concat([df, tmp], axis=0, ignore_index=True)
    else:
      c = pycountry.countries.get(name=Country)
      if c is None:
          c = pycountry.countries.get(common_name=Country)
      try:
          tmp = pd.DataFrame(
              [{"Country": Country, "iso_alpha": c.alpha_3, "PFR_average": float(PFR_average) }])
          df = pd.concat([df, tmp], axis=0, ignore_index=True)
      except:
          continue

fig = px.treemap(df, path=['Country'], values='PFR_average',
                  color='PFR_average', hover_data=['iso_alpha'],
                  color_continuous_scale='RdBu',
                  color_continuous_midpoint=np.average(df['PFR_average'], weights=df['PFR_average']))
fig.update_layout(margin = dict(t=50, l=25, r=25, b=25))
fig.show()